## 미래에셋 빅데이터 콘테스트
# 
### 보험금 청구 분류

### 주성분분석을 사용한 차원축소

In [1]:
#라이브러리 불러오기
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
train = pd.read_csv("D:/thon/data/mirae/classification/train.csv")
test = pd.read_csv("D:/thon/data/mirae/classification/test.csv")
train_dummy = pd.read_csv("D:/thon/data/mirae/classification/train_dummy.csv")
test_dummy = pd.read_csv("D:/thon/data/mirae/classification/test_dummy.csv")

In [3]:
#필요없는 항목 삭제
test_dummy.drop(['Unnamed: 0'], axis = 1, inplace = True)
test_dummy.drop(['ID'], axis = 1, inplace = True)
test_dummy.drop(['base_ym'], axis = 1, inplace = True)

train_dummy.drop(['Unnamed: 0'], axis = 1, inplace = True)
train_dummy.drop(['ID'], axis = 1, inplace = True)
train_dummy.drop(['base_ym'], axis = 1, inplace = True)

### PCA 수행

In [4]:
#x, y 구분
x_train = train_dummy.drop(['target'], axis = 1)
y_train = train_dummy['target']

#타깃 변수 문자형으로 변환
y_train = y_train.astype(str)

#특성행렬 표준화
train_features = StandardScaler().fit_transform(x_train)

#99% 분산을 유지하도록 PCA 클래스 객체 생성
pca = PCA(n_components = 0.99, whiten = True)

#pca 수행
train_features_pca = pca.fit_transform(train_features)


In [8]:
#결과 확인
print("원본 특성 개수:", train_features.shape[1])
print("줄어든 특성 개수:", train_features_pca.shape[1])

원본 특성 개수: 126
줄어든 특성 개수: 103


### PCA 결과 feature를 126개에서 103개로 줄일 수 있었음

In [27]:
#표준화 객체 생성
standardizer = StandardScaler()

#로지스틱회귀 객체 생성
logit = LogisticRegression()

#표준화 후 로지스틱 회귀를 실행하는 파이프라인 생성
pipeline = make_pipeline(standardizer, logit)

#k-fold 교차검증 만들기
kf = KFold(n_splits = 10, shuffle = True, random_state = 1)

#k-fold CV 수행
t1_cv_results = cross_val_score(pipeline, #파이프라인
                            t1_features_pca,#특성 행렬
                            y_t1, #타깃 벡터
                            cv = kf,      #교차검증 기법
                            scoring = 'accuracy', #평가 지표
                            n_jobs = -1)   #모든 CPU코어 사용

In [29]:
t1_cv_results.mean()

0.8929167874259167

### CV 회귀분석 결과 정확도 89.3%